In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [3]:
# Load the MobileNetV2 model without the top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


<ipython-input-3-d76a9584f6b7>:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
# Add global average pooling and a dense layer for CIFAR-10 classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)  # 10 classes in CIFAR-10

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)


In [7]:
# Freeze all the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with Adam optimizer
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [8]:
#Data augmentation helps improve the model's performance by generating additional training data.
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Compute quantities required for featurewise normalization
datagen.fit(train_images)

In [9]:
# Train the model
model.fit(datagen.flow(train_images, train_labels, batch_size=64),
          epochs=10,
          validation_data=(test_images, test_labels))


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


782/782 ━━━━━━━━━━━━━━━━━━━━ 112s 133ms/step - accuracy: 0.2779 - loss: 1.9886 - val_accuracy: 0.3307 - val_loss: 1.8525
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 150s 143ms/step - accuracy: 0.3321 - loss: 1.8379 - val_accuracy: 0.3376 - val_loss: 1.8263
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 141s 143ms/step - accuracy: 0.3400 - loss: 1.8112 - val_accuracy: 0.3459 - val_loss: 1.8091
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 103s 131ms/step - accuracy: 0.3498 - loss: 1.7966 - val_accuracy: 0.3543 - val_loss: 1.7913
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 149s 141ms/step - accuracy: 0.3513 - loss: 1.7764 - val_accuracy: 0.3548 - val_loss: 1.7829
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 103s 131ms/step - accuracy: 0.3562 - loss: 1.7689 - val_accuracy: 0.3576 - val_loss: 1.7841
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 150s 141ms/step - accuracy: 0.3624 - loss: 1.7616 - val_accuracy: 0.3584 - val_loss: 1.7746
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 112s 142ms/step - accuracy: 0.3628 - loss: 1.74

In [10]:
# Unfreeze the last 20 layers
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training the model
model.fit(datagen.flow(train_images, train_labels, batch_size=64),
          epochs=10,
          validation_data=(test_images, test_labels))


Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 189s 227ms/step - accuracy: 0.2902 - loss: 6.6597 - val_accuracy: 0.2101 - val_loss: 2.5804
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 209s 236ms/step - accuracy: 0.4172 - loss: 1.7451 - val_accuracy: 0.3122 - val_loss: 1.9738
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 172s 220ms/step - accuracy: 0.4778 - loss: 1.5017 - val_accuracy: 0.4450 - val_loss: 1.6215
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 172s 219ms/step - accuracy: 0.5186 - loss: 1.3781 - val_accuracy: 0.5197 - val_loss: 1.4352
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 201s 219ms/step - accuracy: 0.5421 - loss: 1.3093 - val_accuracy: 0.5438 - val_loss: 1.3122
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 214s 234ms/step - accuracy: 0.5554 - loss: 1.2668 - val_accuracy: 0.5764 - val_loss: 1.2320
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 173s 220ms/step - accuracy: 0.5730 - loss: 1.2187 - val_accuracy: 0.5952 - val_loss: 1.1978
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 173s 221ms/step - accuracy: 0.5857 -